## **Combine Internal Data with Web Data**

### **Introduction**

In this tutorial, you'll discover how to build a **hybrid agent** that combines real-time web data with your own internal knowledge sources. For enterprises, the true power of agents comes from integrating proprietary internal data with information from the public web.

By the end, you'll be able to:
- **Integrate** web search results and private documents into a single agent workflow
- **Enable** your agent to compare, contrast and synthesize information from both public and internal sources
- **Build** more accurate, up-to-date, and context-aware applications for research, analysis, and decision-making

> **Why is this approach valuable?**
> 
> It empowers your AI to answer questions with the most current information avaiable online, while also leveraging your proprietary data - just like a human expert would.
>
> You'll unlock new capabilities for market research, competitive intelligence, and any scenario where both internal and external knowledge matter.

### **Setting Up**

In [1]:
# install dependencies
!uv add tavily-python langchain-google-genai langchain-chroma langchain langchain-tavily langgraph python-dotenv --quiet

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")
gg_api_key = os.getenv("GOOGLE_API_KEY")

if tavily_api_key is None or gg_api_key is None:
    print("Please set up you API_KEY in .env file")
else:
    print("Get API_KEY successfully")

Get API_KEY successfully


In [3]:
from tavily import TavilyClient
from langchain_google_genai import ChatGoogleGenerativeAI

tavily_client = TavilyClient(api_key=tavily_api_key)
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash",
                             api_key=gg_api_key)

# Test llm setup
response = llm.invoke(input="Hi")
print(response.content)

Hi there! How can I help you today?


### **Vector Database Set Up**

In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# Initialize the embedding model
embedding_model = GoogleGenerativeAIEmbeddings(api_key=gg_api_key, model="text-embedding-004")

# Initialize the Chroma vector store
vector_store = Chroma(
    collection_name="crm",
    embedding_function=embedding_model,
    
)